# KB-BERT

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv, find_dotenv
import sys
import torch
from torch import nn

sys.path.append(os.path.dirname(find_dotenv()))

In [ ]:
#Import the file_handler.py file
from py_scripts.file_handler import read_csv_file

#Read the data
X, Y = read_csv_file("clean.csv")

In [ ]:
import py_scripts.ner_util.ner_system as ner_util

## Getting the data

In [ ]:
#Load data 
X, Y = read_csv_file("clean.csv")

In [ ]:
from sklearn.model_selection import train_test_split

random_state=104

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=random_state, test_size=0.2, shuffle=True)

X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5,random_state=random_state,shuffle=True) 


In [ ]:
print(len(X_test))

## Defining the model

In [ ]:
from transformers import AutoTokenizer, AutoModel

kb_tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')
kb_model = AutoModel.from_pretrained('KB/bert-base-swedish-cased')

In [ ]:
class KB_BERT(nn.Module):
    def __init__(self, seq_labeler):
        super().__init__() 

        p = seq_labeler.params
        self.bert = kb_model

        # Output unit.
        self.top_layer = nn.Linear(self.bert.config.hidden_size, seq_labeler.n_labels)

    def forward(self, words):
        outputs = self.bert(words)
        res = outputs[0]
        return self.top_layer(res)

### Defining NER Parameters

In [ ]:
class NERParameters():

    # Random seed, for reproducibility.
    random_seed = 0
    
    # cuda or cpu
    device = 'mps'    
    
    #Tagging scheme used in data: IO or IOB (Inside-Outside-Beginning)
    tagging_scheme=None
    
    # Training parameters
    n_epochs = 1
    batch_size = 16   
    learning_rate = 0.0001
    weight_decay = 0

    # Word dropout rate.
    word_dropout_prob = 0.0

    bert_max_len = 512

## Finetuning on the data

In [ ]:
ner_system = ner_util.SequenceLabeler(NERParameters(), KB_BERT, bert_tokenizer=AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased'))

ner_system.fit(X_train, Y_train, X_val, Y_val)

## Evaluation of the system

In [ ]:
ner_system.evaluate_model(X_test,Y_test)